# Discover Vector Search API

Find the actual API endpoints for vector-search-service.

In [ ]:
import requests

SERVICE_URL = "http://vector-search-service.servicenow-ai-poc.svc.cluster.local:8000"
print(f"Testing endpoints on: {SERVICE_URL}\n")

## Test Common API Documentation Endpoints

In [ ]:
# FastAPI typically exposes docs
doc_endpoints = [
    "/docs",
    "/redoc",
    "/openapi.json",
]

for endpoint in doc_endpoints:
    try:
        response = requests.get(f"{SERVICE_URL}{endpoint}", timeout=5)
        print(f"{endpoint}: {response.status_code}")
        if response.status_code == 200:
            print(f"  SUCCESS - Try accessing in browser: {SERVICE_URL}{endpoint}")
            if endpoint == "/openapi.json":
                import json
                spec = response.json()
                print(f"\n  Available paths:")
                for path in spec.get('paths', {}).keys():
                    methods = list(spec['paths'][path].keys())
                    print(f"    {path}: {methods}")
    except Exception as e:
        print(f"{endpoint}: ERROR - {e}")

## Test Common Health/Status Endpoints

In [ ]:
health_endpoints = [
    "/",
    "/health",
    "/healthz",
    "/api/health",
    "/api/v1/health",
    "/status",
]

for endpoint in health_endpoints:
    try:
        response = requests.get(f"{SERVICE_URL}{endpoint}", timeout=5)
        print(f"{endpoint}: {response.status_code}")
        if response.status_code == 200:
            print(f"  Response: {response.text[:200]}")
    except Exception as e:
        print(f"{endpoint}: ERROR - {e}")

## Test Collection Endpoints with Different Methods

In [ ]:
# We know this endpoint exists but GET returns 405
# Let's try POST

endpoint = "/api/v1/collections"

print(f"Testing {endpoint} with different HTTP methods:\n")

# Try GET
try:
    response = requests.get(f"{SERVICE_URL}{endpoint}", timeout=5)
    print(f"GET: {response.status_code} - {response.text[:100]}")
except Exception as e:
    print(f"GET: ERROR - {e}")

# Try POST (to list, not create)
try:
    response = requests.post(f"{SERVICE_URL}{endpoint}", json={}, timeout=5)
    print(f"POST (empty): {response.status_code} - {response.text[:100]}")
except Exception as e:
    print(f"POST: ERROR - {e}")

# Try OPTIONS
try:
    response = requests.options(f"{SERVICE_URL}{endpoint}", timeout=5)
    print(f"OPTIONS: {response.status_code}")
    print(f"  Allowed methods: {response.headers.get('Allow', 'N/A')}")
except Exception as e:
    print(f"OPTIONS: ERROR - {e}")

## Test Document Ingestion Endpoint Variations

In [ ]:
# Try different document endpoint patterns
doc_endpoints = [
    "/api/v1/documents",
    "/api/v1/ingest",
    "/api/v1/collections/default/documents",
    "/ingest",
    "/documents",
]

for endpoint in doc_endpoints:
    try:
        # Try POST with minimal payload
        response = requests.post(
            f"{SERVICE_URL}{endpoint}",
            json={"content": "test"},
            timeout=5
        )
        print(f"{endpoint}: {response.status_code}")
        if response.status_code not in [404, 405]:
            print(f"  Response: {response.text[:200]}")
    except Exception as e:
        print(f"{endpoint}: ERROR - {e}")

## Check if there's an API prefix issue

In [ ]:
# Maybe it's /v1/ instead of /api/v1/?
alternative_prefixes = [
    "/v1/collections",
    "/api/collections",
    "/collections",
]

for endpoint in alternative_prefixes:
    try:
        response = requests.get(f"{SERVICE_URL}{endpoint}", timeout=5)
        print(f"{endpoint}: {response.status_code}")
    except Exception as e:
        print(f"{endpoint}: ERROR - {e}")

## Get Service Logs (if oc is available)

In [ ]:
import subprocess

try:
    result = subprocess.run(
        ["oc", "logs", "deployment/vector-search-service", "-n", "servicenow-ai-poc", "--tail=50"],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0:
        print("Recent logs from vector-search-service:")
        print(result.stdout)
    else:
        print("Could not get logs")
        
except FileNotFoundError:
    print("oc command not available")
except Exception as e:
    print(f"Error: {e}")